# ST7 Planification quotidienne d’une équipe mobile
# Phase III

## Importation de modules

In [3]:
# module importation
import numpy as np
from math import ceil
import matplotlib.pyplot as plt

# utilities
from utils import *
from copy import deepcopy

# model classes for employees and nodes
from models_v2 import Employee, Node, Task, Home, Unavail

Déclaration des constantes et des indices :

In [4]:
W = U = T = V = 0
employees = homes = tasks = unavails = nodes = []

## Fonction de lecture de données

In [5]:
def load_data_from_path(path_to_instance: str):
    # load employee data
    Employee.load_excel(path_to_instance)

    # load node data
    Node.clear_previous_data()
    for cls in [Home, Task, Unavail]:
        cls.load_excel(path_to_instance)
    Node.initialize_distance()

    # constants
    global W, U, T, V
    (W, U, T, V) = (Task.count, Unavail.count, Employee.count, Node.count)

    # indices of employees, homes, lunches, tasks, unavailabilities
    global employees, homes, tasks, unavails, nodes
    employees = list(range(T))
    homes = list(range(T))
    tasks = list(range(T, T + W))
    unavails = list((range(T + W, V)))
    nodes = list(range(V))

Lecture de données de test

In [6]:
path_to_test = path_bordeaux_v2 = "./data/InstancesV2/InstanceBordeauxV2.xlsx"
load_data_from_path(path_to_test)

## Implémentation de la classe Solution

In [7]:
from collections import deque

class GreedySolution:

    debug = True

    def __init__(self):
        # the indices of nodes that have not been visited
        self.unvisited_nodes = nodes[:]
        # indices of employees that are not home yet
        self.available_employees = employees[:]
        # employee_nodes[k] is the list of indices of tasks attributed to employee k, in the visit order
        self.employee_nodes = {k: [] for k in employees}
        # unavailabilities that employee should visit
        self.employee_unvisited_unavails = {k: deque(Employee.list[k].unavails) for k in employees}
        # employee_lunch_time[k] is the start of the lunch break of employee k
        self.employee_lunch_time = {k: None for k in employees}
        # the beginning of the task at node i
        self.node_begin_time = {i: None for i in nodes}

    @staticmethod
    def is_lunch_time(time):
        return parse_time_minute("12:00pm") <= time <= parse_time_minute("2:00pm")

    def has_taken_lunch(self, employee_idx: int):
        return self.employee_lunch_time[employee_idx] is not None

    def finish_time(self, employee_idx: int):
        # if employee has finished his work, return infinity
        if employee_idx not in self.available_employees:
            return float("inf")

        # if employee hasn't left home, return his start_time
        if not self.has_not_left_home(employee_idx):
            if self.has_taken_lunch(employee_idx):
                return self.employee_lunch_time[employee_idx] + 60
            return Employee.list[employee_idx].start_time

        # employee's last task's finish time
        last_node_idx = self.employee_nodes[employee_idx][-1]
        last_node_finish_time = self.node_begin_time[last_node_idx] + Node.list[last_node_idx].duration
        if not self.has_taken_lunch(employee_idx) or last_node_finish_time > self.employee_lunch_time[employee_idx]:
            return last_node_finish_time
        return last_node_finish_time

    def has_not_left_home(self, employee_idx):
        return len(self.employee_nodes[employee_idx]) == 0 and not self.has_taken_lunch(employee_idx)

    def has_gone_home(self, employee_idx):
        return len(self.employee_nodes[employee_idx]) != 0 and self.employee_nodes[employee_idx][]

    def current_node(self, employee_idx: int):
        """return index of the employee's current node"""
        # if employee is at home, return home index = employee index
        if employee_idx not in self.available_employees or not self.employee_nodes[employee_idx]:
            return employee_idx
        # else return his last node
        return self.employee_nodes[employee_idx][-1]

    # def has_not_left_home(self, employee_idx):
    #     return employee_idx == self.current_node(employee_idx)

    def employee_distance_to_node(self, employee_idx, node_idx):
        """distance between employee to node at node_idx"""
        return Node.distance[self.current_node(employee_idx), node_idx]

    def travel_time(self, employee_idx, node_idx):
        """time that it takes for employee to go to node in minute"""
        return ceil(self.employee_distance_to_node(employee_idx, node_idx) / Employee.speed)

    def arrival_time(self, employee_idx, node_idx):
        return self.finish_time(employee_idx) + self.travel_time(employee_idx, node_idx)

    def has_enough_time_to_arrive(self, employee_idx, node_idx):
        """determine whether an employee has enough time to go to an unavailability / go home"""
        node = Node.list[node_idx]
        # if node is a task, throw exception
        if node.node_type == "task":
            raise Exception("The node_idx parameter corresponds to the index of a task, expected home or unavailability.")
        if node.node_type == "home":
            return self.finish_time(employee_idx) + self.travel_time(employee_idx, node_idx) <= parse_time_minute("6:00pm")
        # task
        return self.finish_time(employee_idx) + self.travel_time(employee_idx, node_idx) <= node.opening_time

    def next_obstacle(self, employee_idx):
        return self.employee_unvisited_unavails[employee_idx][0] if self.employee_unvisited_unavails[employee_idx] else employee_idx

    def has_time_for_next_obstacle(self, employee_idx):
        # obstacle = next unavailability or home
        return self.has_enough_time_to_arrive(employee_idx, self.next_obstacle(employee_idx))

    def took_lunch_after_last_node(self, employee_idx):
        if not self.has_taken_lunch(employee_idx):
            return False
        if self.has_not_left_home(employee_idx):
            return True
        last_node_idx = self.employee_nodes[employee_idx][-1]
        last_node = Node.list[last_node_idx]
        return self.employee_lunch_time[employee_idx] >= self.node_begin_time[last_node_idx] + last_node.duration

    def take_lunch(self, employee_idx):
        if self.has_taken_lunch(employee_idx):
            raise Exception(f"Employee {employee_idx} has already taken his lunch.")
        self.employee_lunch_time[employee_idx] = self.finish_time(employee_idx)

    def remove_lunch(self, employee_idx):
        self.employee_lunch_time[employee_idx] = None

    def next_possible_operation_time(self, employee_idx, node_idx):
        node = Node.list[node_idx]
        if node.node_type == "home":
            return parse_time_minute("6:00pm")
        if node.node_type == "unavail":
            return node.opening_time
        node: Task
        open_intervals = node.open_intervals()
        arrival_time = self.arrival_time(employee_idx, node_idx)
        for b,e in open_intervals:
            if arrival_time >= b and arrival_time + node.duration <= e:
                return arrival_time
        return float("inf")

    def closest_task(self, employee_idx):
        next_openings = [(node_idx, self.next_possible_operation_time(employee_idx, node_idx)) for node_idx in self.unvisited_nodes]
        node_idx, _ = min(next_openings, key=lambda x,y : y)
        return node_idx

    def visit_node(self, employee_idx, node_idx):
        if self.debug:
            print(f"employee {employee_idx} visits {node_idx}")
        self.unvisited_nodes.remove(node_idx)
        self.employee_nodes[employee_idx].append(node_idx)
        # if employee is going home
        if employee_idx == node_idx:
            self.available_employees.remove(employee_idx)

    def undo_last_task(self, employee_idx):
        if not self.has_not_left_home(employee_idx):
            raise Exception(f"Employee {employee_idx} has not visited any node yet")
        node_idx = self.current_node(employee_idx)
        self.unvisited_nodes.append(node_idx)
        self.employee_nodes[employee_idx].pop()

    def optimize(self):
        while self.available_employees:
            # employee that is available the earliest
            employee_idx = min(self.available_employees, key=lambda k: self.finish_time(k))

            #if the employee has time to visit his next obstacle (ie. unavailability / go home)
            if self.has_time_for_next_obstacle(employee_idx):
                # take lunch if appropriate timing
                if self.is_lunch_time(self.finish_time(employee_idx)) and not self.has_taken_lunch(employee_idx):
                    self.take_lunch(employee_idx)
                # else visit the closest node
                else:
                    closest_task_to_employee = self.closest_task(employee_idx)
                    self.visit_node(employee_idx, node_idx=closest_task_to_employee)

            # if the employee still has time to visit his next obstacle
            if self.has_time_for_next_obstacle(employee_idx):
                continue

            # if the employee can't visit his next obstacle because of the lunch
            if self.took_lunch_after_last_node(employee_idx):
                self.remove_lunch(employee_idx)
                next_obstacle = self.next_obstacle(employee_idx)
                self.visit_node(employee_idx, next_obstacle)
                continue


            self.undo_last_task(employee_idx)
            obstacle_idx = self.next_obstacle(employee_idx)
            self.visit_node(employee_idx, obstacle_idx)

# test

In [12]:
sol = GreedySolution()
sol.optimize()

IndexError: list index out of range

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython_darwin_39_64.pyx", line 1035, in _pydevd_bundle.pydevd_cython_darwin_39_64.PyDBFrame.trace_dispatch
  File "/Users/pingleihe/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/213.6777.50/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "/Users/pingleihe/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/213.6777.50/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "/Users/pingleihe/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/213.6777.50/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [10]:
min([1, 2, 3, 4], key=lambda num: -num)

4